In [1]:
!sudo apt-get update && sudo apt-get install -y espeak
!pip install librosa soundfile transformers torch sequence_align python-dotenv phonemizer
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, Wav2Vec2FeatureExtractor, Wav2Vec2Tokenizer
import torch
import librosa # Import librosa for audio loading and resampling
import soundfile as sf # Still useful for basic audio file checks, though librosa.load can handle it
from sequence_align.pairwise import needleman_wunsch


# audio_file_path = "/content/achyut_pizza_false.wav"

TARGET_SAMPLE_RATE = 16000
model_name = "facebook/wav2vec2-xlsr-53-espeak-cv-ft"

feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name)
tokenizer = Wav2Vec2Tokenizer.from_pretrained(model_name)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,804 kB]
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,077 kB]
Get:13 http://archive.ubuntu.com/ubuntu 

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/412 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2PhonemeCTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:720: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

In [200]:
from dotenv import load_dotenv
from phonemizer.punctuation import Punctuation
from phonemizer.backend import EspeakBackend
from phonemizer.separator import Separator
from sequence_align.pairwise import needleman_wunsch


load_dotenv()
from openai import OpenAI

client = OpenAI(api_key='sk-')
def transcribe_audio(path):
    audio_file= open(path, "rb")

    transcription = client.audio.transcriptions.create(
        model="whisper-1",
        file=audio_file,
        prompt="matlab, jaise ki, vagera-vagera, I'm like,you know what I mean, kind of, um, ah, huh, and so, so um, uh, and um, like um, so like, like it's, it's like, i mean, yeah, ok so, uh so, so uh, yeah so, you know, it's uh, uh and, and uh, like, kind",
    )

    return transcription.text

from text_unidecode import unidecode

def generate_reference_phoneme(reference_text):
    # Clean punctuation and lowercase
    text = unidecode(reference_text)  # handles accents safely
    cleaned = ''.join(c if c.isalnum() or c.isspace() else ' ' for c in text)
    ref_words = [w.lower() for w in cleaned.strip().split() if w]

    backend = EspeakBackend('en-us')
    separator = Separator(phone='', word=' ')  # important for word separation

    # Batch phonemize all words
    phonemes = backend.phonemize(ref_words, separator=separator, strip=True)

    # Zip together
    lexicon = list(zip(ref_words, phonemes))

    return lexicon, ref_words

def expected_phonetics(path,transcript=None):
    if not transcript:
        transcript = transcribe_audio(path=path)

    lexicon, ref_words = generate_reference_phoneme(transcript)
    return transcript, lexicon, ref_words

def genarate_recorded_phonetics(audio_file_path:str):
  # 1. Load your audio file and resample to TARGET_SAMPLE_RATE (16kHz)
  try:
      # librosa.load returns the audio time series and the sampling rate
      audio_input, sample_rate = librosa.load(audio_file_path, sr=TARGET_SAMPLE_RATE, mono=True)
      # By setting sr=TARGET_SAMPLE_RATE, librosa automatically resamples if needed
      # By setting mono=True, librosa automatically converts to mono if needed
  except FileNotFoundError:
      print(f"Error: Audio file not found at {audio_file_path}")
      exit()
  except Exception as e:
      print(f"Error loading or processing audio file: {e}")
      exit()

  input_values = processor(audio_input, return_tensors="pt", sampling_rate=TARGET_SAMPLE_RATE).input_values

  # Retrieve logits
  with torch.no_grad():
      logits = model(input_values).logits

  # Take argmax and decode
  predicted_ids = torch.argmax(logits, dim=-1)
  posteriors = torch.nn.functional.softmax(logits, dim=-1)
  transcription = processor.batch_decode(predicted_ids)
  predicted_ids = torch.argmax(logits, dim=-1)  # shape: [batch, time]

  # Filter each sample in batch
  filtered_ids_batch = []
  filtered_posteriors_batch = []

  for i in range(predicted_ids.size(0)):  # iterate over batch
      ids = predicted_ids[i]
      probs = posteriors[i]

      filtered_ids = []
      filtered_probs = []

      prev_id = -1  # initialize to an invalid ID
      for j in range(ids.size(0)):
          curr_id = ids[j].item()
          if curr_id == 0:
              continue  # skip blank (or pad) token
          if curr_id == prev_id:
              continue  # skip consecutive duplicates
          filtered_ids.append(curr_id)
          filtered_probs.append(probs[j])
          prev_id = curr_id

      filtered_ids_batch.append(torch.tensor(filtered_ids))
      filtered_posteriors_batch.append(torch.stack(filtered_probs))

  # Decode (if needed)
  filtered_transcriptions = processor.batch_decode(
      [ids.tolist() for ids in filtered_ids_batch]
  )

  # Step 2: Convert IDs to phoneme symbols using your index-to-phoneme mapping
  decoded_phonemes = [index_to_phoneme[idx.item()] for idx in filtered_ids_batch[0]]

  return decoded_phonemes,filtered_posteriors_batch[0],filtered_transcriptions

In [201]:
def align_phonetics(seq_a,seq_b):

  aligned_seq_a, aligned_seq_b = needleman_wunsch(
      seq_a,
      seq_b,
      match_score=1.0,
      mismatch_score=-1.0,
      indel_score=-1.0,
      gap="_",
  )
  aligned_reference_seq = ''.join(aligned_seq_a)
  aligned_recorded_seq = ''.join(aligned_seq_b)

  return aligned_reference_seq,aligned_recorded_seq

In [202]:
import pandas as pd

In [203]:
data = pd.read_csv('/content/dataset.csv')

In [204]:
def find_and_add_space(ref_phonemes, recorded_str):
    # print("CALLED",ref_phonemes, recorded_str)
    new_str = ''
    for ix,ch in enumerate(list(ref_phonemes)):
      if ch == ' ':
        new_str += ' '
      else:
        new_str += recorded_str[ix]
    return new_str

In [205]:
# Get the list of phoneme tokens
phoneme_vocab_dict = tokenizer.get_vocab()

# Map: phoneme → ID
special_tokens = [] # ['<s>', '</s>', '<pad>', '|']
phoneme_to_index = {p: i for p, i in tokenizer.get_vocab().items() if p not in special_tokens}

# Map: ID → phoneme (optional, for decoding)
index_to_phoneme = {idx: phoneme for phoneme, idx in phoneme_vocab_dict.items()}

In [ ]:
import torch
import math

def compute_gop_scores(posteriors, aligned_reference_list, threshold=-0.5):
    """
    Computes GOP scores for each time-aligned phoneme.

    Args:
        posteriors (torch.Tensor): Softmax probabilities of shape [T, V]
        aligned_reference_list (List[str]): Ground-truth aligned phonemes (length T)
        phoneme_to_index (Dict[str, int]): Mapping from phoneme symbol to vocabulary index
        threshold (float): GOP threshold to flag mispronunciations (default: -0.5)

    Returns:
        gop_results (List[Tuple[int, str, float, bool]]): List of (t, phoneme, gop_score, is_flagged)
    """
    gop_results = []

    time_steps = posteriors.shape[0]
    for t in range(time_steps):
        ref_phoneme = aligned_reference_list[t]

        if ref_phoneme not in phoneme_to_index:
            if ref_phoneme == ' ' or ref_phoneme == 'ː' or reference_phoneme == '_':
                continue

            print(f"[Warning] Unknown phoneme at t={t}: '{ref_phoneme}' → Skipped")
            continue

        idx_ref = phoneme_to_index[ref_phoneme]
        P_ref = posteriors[t, idx_ref].item()
        P_max = torch.max(posteriors[t]).item()

        if P_ref == 0 or P_max == 0:
            gop = float('-inf')  # To avoid log(0)
        else:
            gop = math.log(P_ref) - math.log(P_max)

        is_flagged = gop < threshold
        gop_results.append((t, ref_phoneme, gop, is_flagged))

    return gop_results

In [228]:
def build_phoneme_to_word_map(reference_text, lexicon):
    """
    Builds a mapping from each phoneme index to its corresponding word using the lexicon.
    Returns a list: word_for_phoneme[t] = word that phoneme at t belongs to
    """
    word_list = reference_text.strip().lower().split()
    lexicon_dict = dict(lexicon)

    word_for_phoneme = []

    for word in word_list:
        if word not in lexicon_dict:
            raise ValueError(f"Word '{word}' not found in lexicon.")
        phonemes = lexicon_dict[word].split()
        word_for_phoneme.extend([word] * len(phonemes))

    return word_for_phoneme

from collections import defaultdict

def get_word_level_mispronunciations(gop_scores, word_for_phoneme):
    """
    Returns a dict: word -> list of (phoneme, gop, is_flagged), and flagged words
    """
    word_gop_dict = defaultdict(list)

    for (t, phoneme, gop, is_flagged) in gop_scores:
        if t >= len(word_for_phoneme):
            continue  # safety check
        word = word_for_phoneme[t]
        word_gop_dict[word].append((phoneme, gop, is_flagged))

    flagged_words = []
    for word, phoneme_scores in word_gop_dict.items():
        if any(is_flagged for _, _, is_flagged in phoneme_scores):
            flagged_words.append(word)

    return word_gop_dict, flagged_words

In [258]:
base_directory = '/content/'
reference_phoneme = ''
phonetics = ''
posteriors = ''
lexicon, ref_words = None,None

def fun():
  for ix,i in data.iterrows():
    print(i['case'])

    path = base_directory + i['case']
    transcript = i['transcript']
    transcript,lexicon, ref_words = expected_phonetics(path=path,transcript=transcript)
    # Clean punctuation and lowercase
    transcript = unidecode(transcript)  # handles accents safely
    transcript = ''.join(c if c.isalnum() or c.isspace() else ' ' for c in transcript)

    reference_phoneme =' '.join([phon for w, phon in lexicon])
    phonetics,posteriors,_ = genarate_recorded_phonetics(path)

    aligned_reference_seq,aligned_recorded_seq = align_phonetics(reference_phoneme,phonetics)
    spaced_recorded_seq = find_and_add_space(aligned_reference_seq,aligned_recorded_seq)

    aligned_recorded_list = list(aligned_recorded_seq)
    aligned_reference_list = list(aligned_reference_seq)

    gop_scores = compute_gop_scores(posteriors,aligned_reference_list)

    print(gop_scores)

    num_flagged = sum(1 for _, _, _, f in gop_scores if f)
    print(f"\n🔎 Total flagged phonemes: {num_flagged}/{len(gop_scores)}")

In [259]:
fun()

smit_pizza_false.wav
[(0, 'a', -4.842253222349682, True), (1, 'ɪ', -9.93379882396894, True), (3, 'l', -4.746661196517033, True), (4, 'ʌ', -7.914169275530611, True), (5, 'v', -7.51627391267138, True), (7, 'i', -8.422184930579775, True), (9, 'ɾ', -8.527955053688103, True), (10, 'ɪ', -5.018045450382674, True), (11, 'ŋ', -8.26870722307252, True), (13, 'p', -9.084248525748862, True), (14, 'i', -10.132020938320883, True), (16, 't', -4.891850476554018, True), (17, 's', -7.299543873655274, True), (18, 'ə', -7.374304246551624, True), (20, 'w', -6.721116555820098, True), (21, 'a', -8.894101042243282, True), (22, 'ɪ', -1.6356486759510513, True), (23, 'l', -8.538440742514151, True), (25, 'w', -11.892970116098478, True), (26, 'ɑ', -9.039323809465884, True), (28, 't', -3.5947529574512123, True), (29, 'ʃ', -8.166454291711506, True), (30, 'ɪ', -9.119105350225391, True), (31, 'ŋ', -6.451827045571603, True), (33, 'd', -4.193107633990808, True), (34, 'ɑ', -8.581684099136865, True), (36, 'k', -5.745254938

In [212]:
len(phonetics)

44

In [218]:
transcript[0]

'aɪloːuːtɪŋbizɐwaɪlwɔtʃɪŋdɔkiməntidʒɐbɔtɡɹuːɾəlɪndiɐ'

In [217]:
''.join(phonetics)

'aɪloːuːtɪŋbizɐwaɪlwɔtʃɪŋdɔkiməntidʒɐbɔtɡɹuːɾəlɪndiɐ'

In [211]:
posteriors.shape

torch.Size([44, 392])

In [177]:
''.join(phonetics)

'aɪloːuːtɪŋbiizɐwaɪllwɔtʃɪŋddɔkimənttidʒɐbɔtɡɹuːɾəlɪnndiɐ'

In [178]:
len(phonetics),len(transcript[0])

(49, 52)

In [175]:
transcript[0]

'aɪloːuːtɪŋbizɐwaɪlwɔtʃɪŋdɔkimənttidʒɐbɔtɡɹuːɾəlɪndiɐ'

In [156]:
for i in phonetics:
  if i != '<pad>':
    print(i,end=' ')

aɪ l oː uː t ɪ ŋ b i i z ɐ w aɪ l l w ɔ tʃ ɪ ŋ d d ɔ k i m ə n t t i dʒ ɐ b ɔ t ɡ ɹ uː ɾ ə l ɪ n n d i ɐ 

In [159]:
phoneme_to_index['<pad>']

0

In [154]:
print(phonetics)

['<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', 'aɪ', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', 'l', '<pad>', '<pad>', 'oː', '<pad>', '<pad>', '<pad>', '<pad>', '

In [91]:
aligned_reference_seq,aligned_recorded_seq,reference_phoneme,phonetics,posteriors,lexicon, ref_words , aligned_reference_list= fun() # scores

52 torch.Size([429, 392])


In [92]:
compute_gop_scores(posteriors,aligned_reference_list)

[Warning] Unknown phoneme at t=2: ' ' → Skipped
[Warning] Unknown phoneme at t=6: ' ' → Skipped
[Warning] Unknown phoneme at t=8: 'ː' → Skipped
[Warning] Unknown phoneme at t=12: ' ' → Skipped
[Warning] Unknown phoneme at t=15: 'ː' → Skipped
[Warning] Unknown phoneme at t=19: ' ' → Skipped
[Warning] Unknown phoneme at t=24: ' ' → Skipped
[Warning] Unknown phoneme at t=27: 'ː' → Skipped
[Warning] Unknown phoneme at t=32: ' ' → Skipped
[Warning] Unknown phoneme at t=35: 'ː' → Skipped
[Warning] Unknown phoneme at t=39: 'ː' → Skipped
[Warning] Unknown phoneme at t=48: ' ' → Skipped
[Warning] Unknown phoneme at t=54: ' ' → Skipped
[Warning] Unknown phoneme at t=61: ' ' → Skipped


IndexError: list index out of range

In [32]:
for i in list(aligned_recorded_seq):
  try:
    print(phoneme_to_index[i],end=' ')
  except:
    if (i==' '):
      print("<SPACE>")
    else:
      print(f"_{i}_",end='\n')

In [20]:
aligned_reference_seq,aligned_recorded_seq,reference_phoneme,phonetics,posteriors,lexicon, ref_words

('aɪ lʌv iːɾɪŋ piːtsə waɪl wɑːtʃɪŋ dɑːkjuːmɛntɚɹiz ɐbaʊt ɹʊɹɹəl ɪndɪə',
 '',
 'aɪ lʌv iːɾɪŋ piːtsə waɪl wɑːtʃɪŋ dɑːkjuːmɛntɚɹiz ɐbaʊt ɹʊɹɹəl ɪndɪə',
 'aɪloːuːtɪŋbizɐwaɪlwɔtʃɪŋdɔkimənttidʒɐbɔtɡɹuːɾəlɪndiɐ',
 tensor([[9.9989e-01, 5.8406e-11, 5.3798e-11,  ..., 7.4683e-11, 9.4737e-11,
          6.6592e-11],
         [9.9993e-01, 2.8469e-11, 2.6531e-11,  ..., 3.7413e-11, 4.6658e-11,
          3.3853e-11],
         [9.9994e-01, 2.9434e-11, 2.7557e-11,  ..., 4.0384e-11, 4.9482e-11,
          3.6153e-11],
         ...,
         [9.9994e-01, 1.5462e-11, 1.2954e-11,  ..., 2.2260e-11, 2.2036e-11,
          1.9597e-11],
         [9.9995e-01, 1.3690e-11, 1.1219e-11,  ..., 1.9451e-11, 2.0240e-11,
          1.7461e-11],
         [9.9992e-01, 2.5232e-11, 2.1444e-11,  ..., 3.5883e-11, 3.5969e-11,
          2.9974e-11]]),
 [('i', 'aɪ'),
  ('love', 'lʌv'),
  ('eating', 'iːɾɪŋ'),
  ('pizza', 'piːtsə'),
  ('while', 'waɪl'),
  ('watching', 'wɑːtʃɪŋ'),
  ('documentaries', 'dɑːkjuːmɛntɚɹiz'),
  ('about', 'ɐba

In [73]:
lexicon

[('i', 'aɪ'),
 ('love', 'lʌv'),
 ('eating', 'iːɾɪŋ'),
 ('pizza', 'piːtsə'),
 ('while', 'waɪl'),
 ('watching', 'wɑːtʃɪŋ'),
 ('documentaries', 'dɑːkjuːmɛntɚɹiz'),
 ('about', 'ɐbaʊt'),
 ('rural', 'ɹʊɹɹəl'),
 ('india', 'ɪndɪə')]

In [75]:
aligned_reference_seq,

('aɪ lʌv iːɾɪŋ piːtsə waɪl wɑːtʃɪŋ dɑːkjuːmɛntɚɹiz ɐbaʊt ɹʊɹɹəl ɪndɪə',
 'aɪ l_o uːtɪŋ bi___z waɪl w_ɔtʃɪŋ d_ɔk__imənt_tid ɐb_ɔt ɹuːɾəl ɪndiɐ')

In [44]:
posteriors.shape

torch.Size([429, 392])